# Cross-sectional Predictability

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import urllib.parse
from sqlalchemy import create_engine
!pip install pymssql


[notice] A new release of pip available: 22.2.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
chars = [
    "bm",
    "ep",
    "cashpr",
    "dy",
    "lev",
    "sp",
    "roic",
    "rd_sale",
    "rd_mve",
    "agr",
    "gma",
    "chcsho",
    "lgr",
    "acc",
    "pctacc",
    "cfp",
    "absacc",
    "age",
    "chinv",
    "hire",
    "sgr",
    "pchsale_pchinvt",
    "pchsale_pchrect",
    "pchgm_pchsale",
    "pchsale_pchxsga",
    "depr",
    "pchdepr",
    "invest",
    "egr",
    "grcapx",
    "tang",
    "sin",
    "currat",
    "pchcurrat",
    "quick",
    "pchquick",
    "salecash",
    "salerec",
    "saleinv",
    "pchsaleinv",
    "cashdebt",
    "realestate",
    "divi",
    "divo",
    "securedind",
    "secured",
    "convind",
    "grltnoa",
    "rd",
    "operprof",
    "ps",
    "chpmia",
    "chatoia",
    "chempia",
    "bm_ia",
    "pchcapx_ia",
    "tb",
    "cfp_ia",
    "mve_ia",
    "herf",
    "orgcap",
    "mve",
    "chtx",
    "roaq",
    "roeq",
    "rsup",
    "stdacc",
    "roavol",
    "stdcf",
    "cash",
    "cinvest",
    "nincr",
    "sue",
    "aeavol",
    "ear",
    "ms",
    "disp",
    "chfeps",
    "fgr5yr",
    "nanalyst",
    "sfe",
    "chnanalyst",
    "mom6m",
    "mom12m",
    "mom36m",
    "mom1m",
    "dolvol",
    "chmom",
    "turn",
    "ipo",
    "indmom",
    "maxret",
    "retvol",
    "baspread",
    "std_dolvol",
    "std_turn",
    "ill",
    "zerotrade",
    "beta",
    "betasq",
    "pricedelay",
    "idiovol",
]

intchars = [
    "age",
    "sin",
    "divi",
    "divo",
    "securedind",
    "convind",
    "rd",
    "ps",
    "nincr",
    "ms",
    "ipo",
]

chars = [x for x in chars if x not in intchars]
chars = np.sort(chars)
labels = ["Lo 20", "Qnt 2", "Qnt 3", "Qnt 4", "Hi 20"]

server = "eu-az-sql-serv1.database.windows.net:1433"
database = "dgn022k6348dcyh"
username = "uhgrque4d8p77hf"
password = "FfWrgFcK$Vnk@9BAgKH4nbEDF"
password = urllib.parse.quote_plus(password)

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database
conn = create_engine(string).connect()


Pull the data from SQL server

In [3]:
char = 'mve, beta, mom12m, bm'        # example characteristics

df = pd.read_sql(
    " select ticker, date, ret, " + char + " from ghz where date>='2000-01-01' ", conn
)
df = df.dropna()

In [4]:
df

,ticker,date,ret,mve,beta,mom12m,bm
0,CELG,2000-04-28,0.418079,14.573799,1.292682,9.343750,-0.014615
1,APW,2000-04-28,0.004386,13.922989,1.242077,-0.110433,0.350028
2,ACO,2000-04-28,0.065041,12.835614,0.943564,0.604415,0.651691
3,XRAY,2000-04-28,0.024229,14.204507,0.612569,0.109957,0.305512
4,DCN,2000-04-28,0.077605,15.342823,1.174181,-0.415371,0.433959
...,...,...,...,...,...,...,...
470930,BOFI,2017-04-28,-0.085725,14.320145,1.653209,0.477976,0.606040
470931,PAY,2017-04-28,-0.010144,14.552937,1.760381,-0.268059,0.264849
470932,DXCM,2017-04-28,-0.079901,15.805055,1.381914,0.150935,0.033058
470933,MORN,2017-04-28,-0.066667,15.031721,0.681260,-0.080696,0.183465


In [6]:
len(df.index)
df.date.min()

datetime.date(2000, 1, 31)

## Cross-sectional Method #1: Sorting

In [7]:
# Sorting function
def cut_quintiles(x):
    try:
        out = pd.qcut(x, 5, labels=["Lo 20", "Qnt 2", "Qnt 3", "Qnt 4", "Hi 20"])
    except:
        out = pd.Series(np.nan, index=x.index)
    return out

In [42]:
# Characteristic to sort on
# CHAR = 'beta'
# CHAR = 'mve'
CHAR = 'bm'

df["quintile"] = df.groupby("date")[CHAR].apply(cut_quintiles)

In [44]:
# Check to see we have about the same number of observations in each bin
df.quintile.value_counts()

Lo 20    89719
Hi 20    89666
Qnt 2    89577
Qnt 4    89577
Qnt 3    89551
Name: quintile, dtype: int64

In [45]:
# Time-series of EW average portfolio across stocks
df = df.dropna(subset=["quintile"])
rets = df.groupby(["date", "quintile"]).ret.mean()
rets.head(10).round(4)

date        quintile
2000-01-31  Lo 20      -0.0224
            Qnt 2      -0.0129
            Qnt 3      -0.0364
            Qnt 4      -0.0324
            Hi 20      -0.0334
2000-02-29  Lo 20       0.1881
            Qnt 2       0.1488
            Qnt 3       0.0568
            Qnt 4       0.0329
            Hi 20       0.0660
Name: ret, dtype: float64

In [46]:
rets = rets.unstack()
rets.head(2)

quintile,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20
date,,,,,
2000-01-31,-0.022418,-0.012871,-0.036378,-0.032441,-0.033444
2000-02-29,0.188089,0.148843,0.056754,0.032929,0.066006


In [47]:
# Time-series average of each portfolio
mns = rets.mean()
print(f'Sorting on {CHAR}:\nAverage returns:\n{mns.round(4)}')

hilo = mns['Hi 20']-mns['Lo 20']
print(f'\nHi-Lo  average return is: {hilo: .2%}')

Sorting on bm:
Average returns:
quintile
Lo 20    0.0091
Qnt 2    0.0101
Qnt 3    0.0104
Qnt 4    0.0108
Hi 20    0.0109
dtype: float64

Hi-Lo  average return is:  0.19%


## Cross-sectional Method #2: Regression

Example #1: Single characteristic

In [48]:
# Run a cross-sectional regression for each date
coefs = df.groupby('date').apply(lambda d: sm.OLS(d.ret,sm.add_constant(d[CHAR])).fit().params) 
coefs

,const,bm
date,,
2000-01-31,-0.021253,-0.014922
2000-02-29,0.143935,-0.108529
2000-03-31,0.006210,0.068091
2000-04-28,-0.083208,0.078237
2000-05-31,-0.055815,0.049750
...,...,...
2021-08-31,0.015652,0.009185
2021-09-30,-0.046549,0.040901
2021-10-29,0.045063,0.000656


In [49]:
coefs.mean()
T = len(coefs[CHAR])
tsreg = sm.OLS(coefs[CHAR],np.ones(T)).fit()

In [50]:
print(tsreg.summary())

                            OLS Regression Results                            
Dep. Variable:                     bm   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 23 Mar 2023   Prob (F-statistic):                nan
Time:                        14:29:34   Log-Likelihood:                 510.17
No. Observations:                 264   AIC:                            -1018.
Df Residuals:                     263   BIC:                            -1015.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0009      0.002      0.400      0.6

In [41]:
# Plot coefficient on beta
import plotly.graph_objects as go
trace  = go.Scatter(x=coefs.index, y=coefs[CHAR], mode="lines")
fig = go.Figure()
fig.add_trace(trace)
fig.update_yaxes(title=f'Coefficient on {CHAR}',tickformat=".4f")
fig.update_xaxes(title='Date')
fig.show()


Example #2: Multiple characteristics

In [51]:
CHARS = ['beta','mve', 'bm','mom12m']
coefs = df.groupby('date').apply(lambda d: sm.OLS(d.ret,sm.add_constant(d[CHARS])).fit().params) 

In [52]:
stats = pd.DataFrame(dtype=float,index=coefs.columns,columns=['mean','tstat','pval'])
for coef in coefs.columns :
    tsreg = sm.OLS(coefs[coef],np.ones(T)).fit()

    stats.loc[coef,'mean'] = coefs[coef].mean()
    stats.loc[coef,'tstat']= tsreg.tvalues['const']
    stats.loc[coef,'pval'] = tsreg.pvalues['const']
stats.round(4)

,mean,tstat,pval
const,0.0269,3.3762,0.0008
beta,-0.0016,-0.5857,0.5586
mve,-0.0011,-2.2309,0.0265
bm,-0.0009,-0.5383,0.5909
mom12m,-0.0004,-0.1621,0.8713


In [53]:
# Plot a chosen characteristic's coefficients over time
coef = 'mom12m'
trace  = go.Scatter(x=coefs.index, y=coefs[coef], mode="lines")
fig = go.Figure()
fig.add_trace(trace)
fig.update_yaxes(title='Coefficient on '+coef,tickformat=".4f")
fig.update_xaxes(title='Date')
fig.show()